In [1]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import xml.etree.ElementTree as et

import xmltodict


import json
import re

from IPython.display import display
from IPython.core.display import display, HTML
from tqdm.notebook import tqdm



from geopy.geocoders import Nominatim
import folium

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24828\1671522653.py:15: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>.output_result { max-width:200% !important; }</style>"))
display(HTML("<style>.output_result { max-height:200% !important; }</style>"))

display(HTML("<style>.prompt { display:none !important; }</style>"))

In [2]:
def url_split(url):
    main_,para_1 = url.split("?")
    para_2 = para_1.split("&")
    para_3 = [d.replace("=",":").split(":") for d in para_2 if len(d.replace("=",":").split(":")) != 1]
    params_ = {}
    #print(para_3)
    for key,value in para_3:
        params_[key] = value
    return main_,params_

In [3]:
def query_df_out(query_df,query,version=None):
    a = query_df[query_df["구분"]==query]
    if version =="all":
        b = a
    elif version == 1:
        b = a[query_df["항목구분"]==0]
    else:
        b = a[query_df["항목구분"]==1]
    return b

In [4]:
df_in = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\Query_Book.xlsx",sheet_name="all")
df_out = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\Query_Book.xlsx",sheet_name="out")
df_item = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\Query_Book.xlsx",sheet_name="items")

## Making dictionary

In [5]:
df_in2 = df_in[["항목명(영문)","항목명(국문)"]].drop_duplicates(subset=["항목명(영문)"]).reset_index(drop=True)
df_out2 = df_out[["항목명(영문)","항목명(국문)"]].drop_duplicates(subset=["항목명(영문)"]).reset_index(drop=True)

In [6]:
cate_ = {}
for row,col in df_in2.iterrows():
    key_ = col["항목명(영문)"].replace("\\","")
    value_  =col["항목명(국문)"].replace("\\","")
    cate_[key_] = value_

for row,col in df_out2.iterrows():
    key_ = col["항목명(영문)"].replace("\\","")
    value_  =col["항목명(국문)"].replace("\\","")
    cate_[key_] = value_

In [7]:
url_main = "http://apis.data.go.kr/B460014/foodBankInfoService2"

In [8]:
encode = "%2Fxc7f1IdclYG5tj6DIg1cU51nehX%2FzyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA%3D%3D"
decode = "/xc7f1IdclYG5tj6DIg1cU51nehX/zyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA=="

In [9]:
query_1 = {
    "getUserInfo": "이용자", "getFcltyGrpInfo" : "이용시설단체", "getCntrbtrInfo":"기부자",
    "getRceptStat": "접수현황", "getProvdStat": "제공현황",
    "getCnttgInfo":"물품현황","getSpctrInfo":"센터통계", "getPreferInfo":"선호물품"
}

In [10]:
query_1_2 = {
    value:key for key,value in query_1.items()
}

In [11]:
df_item[df_item["구분"]=="rceptSpctrCd"]

,코드,명,구분
1076,S00101,전국푸드뱅크,rceptSpctrCd
1077,S00201,서울시광역푸드뱅크,rceptSpctrCd
1078,S00301,영등포사랑나눔푸드뱅크,rceptSpctrCd
1079,S00401,성동희망푸드뱅크,rceptSpctrCd
1080,S00501,종로구푸드뱅크,rceptSpctrCd
...,...,...,...
1721,S70301,새희망푸드뱅크,rceptSpctrCd
1722,S70401,참좋은푸드마켓,rceptSpctrCd
1723,S70501,동군산푸드뱅크,rceptSpctrCd
1724,S70601,전주시새샘푸드뱅크,rceptSpctrCd


# Date 별 지원센터 조사

In [12]:
query_1_2["센터통계"]

'getSpctrInfo'

In [13]:
url_test = url_main + "/" + "getSpctrInfo" + "?serviceKey=" + encode

In [14]:
paras_ = {
        "stdrYm": 0,
    "numOfRows": 0,
    "pageNo": 0

}

In [15]:
paras_["stdrYm"] = 201701
paras_["numOfRows"] = 1000
paras_["pageNo"] = 1

for key,value in paras_.items():
    url_test += "&" + key + "=" + str(value)

In [16]:
html = requests.get(url_test).text

In [69]:
result = xmltodict.parse(html)
dictionay = json.loads(json.dumps(result))["response"]["body"]["items"]["item"]
out = pd.DataFrame(dictionay)

In [70]:
for col_ in out.columns:
    target_ = df_item[df_item["구분"]==col_]
    dic_tar = {col["코드"]:col["명"] for row,col in target_.iterrows()}
    col_list = []
    if len(target_)!=0:
        try:
            data_ = out[col_].map(lambda x: int(x))#.map(lambda x: x[1:] if x[0]=="0" else())
        except:
            data_ = out[col_]
        try:
            data_2 = [dic_tar[i] for i in data_.values]
        except:
            pass
        out[col_] = data_2
    else:
        pass

In [71]:
out.head()

,stdrYm,areaCd,unitySignguCd,spctrUncd,spctrSecd,spctrCd,spctrTelno,spctrAdres,spctrDetailAdres,spctrStscd,operMbyLclasCd,operMbySclasCd,sttemntSeccd,registDe,rceptAmt,provdAmt,undtakeAmt,trnsferAmt,userCo
0,201701,중앙,중앙,전국,푸드뱅크,전국푸드뱅크,02-2077-3985,서울특별시 마포구 만리재로 14,한국사회복지회관 4층,사용,법인 운영,사회복지법인,당연신고,20020520,1502818406.3,0,0,1518214594,0
1,201701,서울,서울,광역,푸드뱅크,서울시광역푸드뱅크,02-905-1377,서울 도봉구 창동,1-7,사용,법인 운영,사회복지법인,당연신고,20001205,214050197,14366322.44,81396821,665502574,652
2,201701,서울,서울,기초,푸드뱅크,영등포사랑나눔푸드뱅크,02-846-1377,서울특별시 영등포구 도림로61길 7(대림동),7,사용,법인 운영,사회복지법인,당연신고,20090101,184096211,131834005.5,2825560,5288924,190
3,201701,서울,서울,기초,푸드뱅크,성동희망푸드뱅크,02-425-1377,서울특별시 성동구 왕십리로28길 10 준희빌딩(도선동),1층,사용,비영리 민간 단체·시설 운영,비영리 민간 단체·시설,당연신고,20020101,31325960,38783216.15,6467190,21876559,32
4,201701,서울,서울,기초,푸드뱅크,종로구푸드뱅크,02-762-1377,서울 종로구 숭인1동,55-2 숭인1동주민센터 4층,사용,법인 운영,사회복지법인,임의신고,20020101,70196990,62632106.18,24167245,32693612,38


In [72]:
out = out.rename(columns={col:cate_[col] for col in out.columns}).fillna("")
col_tar =  ["기준년월","지역코드","지원센터단위코드","지원센터구분코드","지원센터코드","지원센터주소","지원센터상세주소"]
out = out[col_tar]
out

,기준년월,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터주소,지원센터상세주소
0,201701,중앙,전국,푸드뱅크,전국푸드뱅크,서울특별시 마포구 만리재로 14,한국사회복지회관 4층
1,201701,서울,광역,푸드뱅크,서울시광역푸드뱅크,서울 도봉구 창동,1-7
2,201701,서울,기초,푸드뱅크,영등포사랑나눔푸드뱅크,서울특별시 영등포구 도림로61길 7(대림동),7
3,201701,서울,기초,푸드뱅크,성동희망푸드뱅크,서울특별시 성동구 왕십리로28길 10 준희빌딩(도선동),1층
4,201701,서울,기초,푸드뱅크,종로구푸드뱅크,서울 종로구 숭인1동,55-2 숭인1동주민센터 4층
...,...,...,...,...,...,...,...
438,201701,전남,기초,푸드뱅크,담양군기초푸드뱅크,전라남도 담양군 담양읍 죽향문화로 431-33,
439,201701,서울,기초,푸드뱅크,글로벌푸드누리푸드뱅크,서울특별시 강동구 천호대로162길 66,(성내동)
440,201701,전북,기초,푸드마켓,부안행복나눔푸드마켓,전라북도 부안군 부안읍 번영로 97-1,부안행복나눔푸드마켓
441,201701,제주,기초,푸드뱅크,행복나눔푸드뱅크,제주특별자치도 서귀포시 서문로 30,(서귀동)


In [73]:
region_list = list(out["지역코드"].value_counts().keys())
national = out[out["지원센터단위코드"]=="전국"]
for region in region_list:
    df_ = out[out["지역코드"]==region]
    center = df_[df_["지원센터단위코드"]=="광역"]
    brench = df_[df_["지원센터단위코드"]=="기초"]
    display(center)
    display(brench)
    

,기준년월,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터주소,지원센터상세주소
74,201701,경기,광역,푸드뱅크,경기도광역푸드뱅크,경기도 수원시 장안구 서부로 2161 (율전동),(89-5번지 6층)


,기준년월,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터주소,지원센터상세주소
75,201701,경기,기초,푸드뱅크,수원시푸드뱅크,경기도 수원시 팔달구 매산로 86,(매산로3가)
76,201701,경기,기초,푸드뱅크,성남기초푸드뱅크,경기도 성남시 중원구 성남대로1151번길 20-7,(성남동)
77,201701,경기,기초,푸드뱅크,성남시청솔푸드뱅크,경기도 성남시 분당구 미금로 246,"(금곡동, 청솔마을주공6단지아파트)"
78,201701,경기,기초,푸드뱅크,의정부시푸드뱅크,경기도 의정부시 추동로108번길 90,1층 (신곡동)
79,201701,경기,기초,푸드뱅크,안양시푸드뱅크,경기도 안양시 동안구 동안로 153(비산동),평생교육원 2층
...,...,...,...,...,...,...,...
429,201701,경기,기초,푸드뱅크,나누리푸드뱅크,경기도 의정부시 흥선로16번길 12,(가능동)
431,201701,경기,기초,푸드뱅크,아름다운동행 푸드뱅크,경기도 용인시 처인구 금령로71번길 8,중105호 (김량장동)
432,201701,경기,기초,푸드뱅크,희망등대푸드뱅크,경기도 성남시 수정구 희망로534번길 39,(산성동) 1층
435,201701,경기,기초,푸드뱅크,화성은혜푸드뱅크,경기도 화성시 봉담읍 독정길 30,화성은혜푸드뱅크


,기준년월,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터주소,지원센터상세주소
1,201701,서울,광역,푸드뱅크,서울시광역푸드뱅크,서울 도봉구 창동,1-7


,기준년월,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터주소,지원센터상세주소
2,201701,서울,기초,푸드뱅크,영등포사랑나눔푸드뱅크,서울특별시 영등포구 도림로61길 7(대림동),7
3,201701,서울,기초,푸드뱅크,성동희망푸드뱅크,서울특별시 성동구 왕십리로28길 10 준희빌딩(도선동),1층
4,201701,서울,기초,푸드뱅크,종로구푸드뱅크,서울 종로구 숭인1동,55-2 숭인1동주민센터 4층
5,201701,서울,기초,푸드뱅크,동대문구푸드뱅크,서울특별시 동대문구 천호대로2길 23-9(신설동),
6,201701,서울,기초,푸드뱅크,중랑구푸드뱅크,서울특별시 중랑구 봉화산로56길 22,(신내동)
...,...,...,...,...,...,...,...
401,201701,서울,기초,푸드마켓,성북구기초푸드마켓2호점,서울 성북구 하월곡동,13-7 게이또 빌딩 1층
412,201701,서울,기초,푸드뱅크,노원푸드뱅크,서울특별시 노원구 동일로 1229,(중계동)
419,201701,서울,기초,푸드뱅크,나눔과기쁨203지회푸드뱅크,서울특별시 송파구 마천로31길 22 원인통상(주)(마천동),
439,201701,서울,기초,푸드뱅크,글로벌푸드누리푸드뱅크,서울특별시 강동구 천호대로162길 66,(성내동)


,기준년월,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터주소,지원센터상세주소
107,201701,충북,광역,푸드뱅크,충북광역푸드뱅크,충청북도 청주시 흥덕구 공단로 87,충북종합사회복지센터 3층


...

,기준년월,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터주소,지원센터상세주소
255,201701,세종,기초,푸드뱅크,세종사랑나눔푸드마켓·뱅크,세종특별자치시 조치원읍 세종로 2567,
359,201701,세종,기초,푸드마켓,세종사랑나눔푸드마켓,세종특별자치시 조치원읍 세종로 2567,


,기준년월,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터주소,지원센터상세주소


,기준년월,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터주소,지원센터상세주소


,기준년월,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터주소,지원센터상세주소
378,201701,중앙물류,광역,물류센터,세종중앙물류센터,세종특별자치시 연동면 연청로 745-86,9동


,기준년월,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터주소,지원센터상세주소


# Visualization

### 1. Transfer "post code" into "lat" and "lng"

In [74]:
out["지원센터주소"] = [i.split("(")[0] for i in out["지원센터주소"]]

In [75]:
out["지원센터주소"] = [i.replace("번길","").replace("번지","").strip() for i in out["지원센터주소"]]
out["지원센터상세주소"] = [i.replace("번길","").replace("번지","").strip() for i in out["지원센터상세주소"]]


In [76]:
out["주소"]=out["지원센터주소"] + " " + out["지원센터상세주소"]

In [77]:
reg = re.compile("(.+[0-9]+) \w+")
test_list = []
for i in range(len(out)):
    can_ = out.loc[i]["지원센터주소"]
    test_ = reg.findall(can_)
    if len(test_)==0:
        test_list.append(can_)
    else:
        test_list.extend(test_)

out["지원센터주소2"] = test_list


In [78]:
reg = re.compile("(.+[가-힣]+)([0-9]+.+)")
test_list = []
for i in range(len(out)):
    can_ = out.loc[i]["지원센터주소"]
    test_ = reg.findall(can_)
    if len(test_)==0:
        test_list.append(can_)
    else:
        b_ = test_[0][0] + " " + test_[0][1]
        test_list.append(b_)
out["지원센터주소3"] = test_list

In [79]:
out

,기준년월,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터주소,지원센터상세주소,주소,지원센터주소2,지원센터주소3
0,201701,중앙,전국,푸드뱅크,전국푸드뱅크,서울특별시 마포구 만리재로 14,한국사회복지회관 4층,서울특별시 마포구 만리재로 14 한국사회복지회관 4층,서울특별시 마포구 만리재로 14,서울특별시 마포구 만리재로 14
1,201701,서울,광역,푸드뱅크,서울시광역푸드뱅크,서울 도봉구 창동,1-7,서울 도봉구 창동 1-7,서울 도봉구 창동,서울 도봉구 창동
2,201701,서울,기초,푸드뱅크,영등포사랑나눔푸드뱅크,서울특별시 영등포구 도림로61길 7,7,서울특별시 영등포구 도림로61길 7 7,서울특별시 영등포구 도림로61길 7,서울특별시 영등포구 도림로 61길 7
3,201701,서울,기초,푸드뱅크,성동희망푸드뱅크,서울특별시 성동구 왕십리로28길 10 준희빌딩,1층,서울특별시 성동구 왕십리로28길 10 준희빌딩 1층,서울특별시 성동구 왕십리로28길 10,서울특별시 성동구 왕십리로 28길 10 준희빌딩
4,201701,서울,기초,푸드뱅크,종로구푸드뱅크,서울 종로구 숭인1동,55-2 숭인1동주민센터 4층,서울 종로구 숭인1동 55-2 숭인1동주민센터 4층,서울 종로구 숭인1동,서울 종로구 숭인 1동
...,...,...,...,...,...,...,...,...,...,...
438,201701,전남,기초,푸드뱅크,담양군기초푸드뱅크,전라남도 담양군 담양읍 죽향문화로 431-33,,전라남도 담양군 담양읍 죽향문화로 431-33,전라남도 담양군 담양읍 죽향문화로 431-33,전라남도 담양군 담양읍 죽향문화로 431-33
439,201701,서울,기초,푸드뱅크,글로벌푸드누리푸드뱅크,서울특별시 강동구 천호대로162길 66,(성내동),서울특별시 강동구 천호대로162길 66 (성내동),서울특별시 강동구 천호대로162길 66,서울특별시 강동구 천호대로 162길 66
440,201701,전북,기초,푸드마켓,부안행복나눔푸드마켓,전라북도 부안군 부안읍 번영로 97-1,부안행복나눔푸드마켓,전라북도 부안군 부안읍 번영로 97-1 부안행복나눔푸드마켓,전라북도 부안군 부안읍 번영로 97-1,전라북도 부안군 부안읍 번영로 97-1
441,201701,제주,기초,푸드뱅크,행복나눔푸드뱅크,제주특별자치도 서귀포시 서문로 30,(서귀동),제주특별자치도 서귀포시 서문로 30 (서귀동),제주특별자치도 서귀포시 서문로 30,제주특별자치도 서귀포시 서문로 30


In [80]:
geo_local = Nominatim(user_agent='South Korea')

In [81]:
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

In [83]:
tq = tqdm(range(len(out)))
la_ln = []
for i in tq:
    first_ = out.loc[i]["주소"]
    a_ = geocoding(first_)
    post_ = first_
    if a_[0] == 0:
        second_ = out.loc[i]["지원센터주소"]
        a_ = geocoding(second_)
        post_ = second_
        if a_[0] == 0:
            third = out.loc[i]["지원센터주소2"]
            a_ = geocoding(third)
            post_ = third
            if a_[0]==0:
                fourth = out.loc[i]["지원센터주소3"]
                a_ = geocoding(fourth)
                post_ = fourth
            else:
                pass
        else:
            pass
    else:
        pass
    la_ln.append(a_)
    print(post_,a_)

  0%|          | 0/443 [00:00<?, ?it/s]

서울특별시 마포구 만리재로 14 [37.5440615, 126.951932]
서울 도봉구 창동 1-7 [37.64643, 127.04326]
서울특별시 영등포구 도림로61길 7 7 [37.4983547, 126.9021709]
서울특별시 성동구 왕십리로28길 10 [37.5652258, 127.0318264]
서울 종로구 숭인1동 [37.5776359, 127.01701239299508]
서울특별시 동대문구 천호대로2길 23-9  [37.5737092, 127.024738]
서울특별시 중랑구 봉화산로56길 22 [37.6075225, 127.09938755298236]
서울특별시 강북구 삼각산로 119 [37.6408474, 127.017533]
서울특별시 도봉구 덕릉로 329 [37.64612, 127.0523]
서울특별시 은평구 서오릉로 134-5 [37.6043166, 126.9242405]
서울 강서구 공항동 51-42 [37.5556799, 126.79312568532687]
서울특별시 구로구 구로동로50길 10 [37.4984033, 126.8830369]
서울특별시 금천구 탑골로 35 [37.4520648, 126.91346023283998]
서울특별시 양천구 중앙로 250 [37.5197122, 126.8531251]
서울특별시 관악구 신림로58길 13 [37.4824532, 126.9316895]
서울특별시 서초구 바우뫼로 43 [37.4798659, 127.0410708]
서울특별시 송파구 마천로51길 2 [37.4967357, 127.1521051]
서울특별시 강동구 동남로 926 [37.5575411, 127.1566952]
서울특별시 성북구 삼양로8가길 32 [37.6089458, 127.0245806]
서울 동작구 사당동 252-17 [37.4842484, 126.97605050748652]
서울특별시 마포구 월드컵로 235 [37.5596169, 126.9069064]
서울특별시 용산구 한강대로 345 [37.5447751, 126.

충청북도 보은군 보은읍 보청대로 1907  [36.4891402, 127.7198752]
강원도 강릉시 동부시장3길 22 [37.7590256, 128.9012287]
충북 청주시 흥덕구 운천동 1486 [36.6485352, 127.4754254]
서울특별시 강남구 개포로 605 [37.4919828, 127.0735147708356]
충남 예산군 예산읍 관작리 [0, 0]
경기도 파주시 파주읍 봉서산로 138 [0, 0]
서울 서대문구 연희동 133-1 [37.57234, 126.93084]
전라북도 장수군 장수읍 시장로 27-9 [35.6494394, 127.5182433]
전라북도 부안군 부안읍 번영로 97 [35.72673415, 126.7310634185329]
전라북도 순창군 순창읍 장류로 407-30 [35.3698142, 127.1072195]
경기 양주시 광적면 [37.8245077, 126.9838784]
서울 은평구 불광동 [37.6160647, 126.9339579]
서울 노원구 공릉동 [37.63021515, 127.0872902349954]
충청남도 논산시 부적면 반송 2길 5 [0, 0]
경기 동두천시 광암동 [0, 0]
인천광역시 서구 심곡로 124 10 [37.5421841, 126.6751789]
인천광역시 미추홀구 매소홀로 418 14-57 [0, 0]
강원도 횡성군 횡성읍 어수로 46  [37.4933668, 127.9836573]
강원도 인제군 인제읍 인제로 199 10 [38.0606387, 128.1645001]
제주특별자치도 제주시 중앙로2길 35 [33.5171714, 126.5219029]
강원도 화천군 하남면 춘화로 3341 [38.0374366, 127.7152318]
강원도 양양군 강현면 일출로 42 [0, 0]
충청북도 음성군 음성읍 시장로 90 [36.9368772, 127.6919337]
전라북도 완주군 삼례읍 별산농원길 20  [35.9245394, 127.0914779]
충청남도 천안시 동남구 태조

부산 금정구 남산동 42-5 [35.2680875, 129.0897851]
부산 영도구 동삼동 [35.0751315, 129.0728955]
부산광역시 사상구 동주로 6-9 [35.147702, 129.00008]
광주 광산구 우산동 1573~1610 [0, 0]
대구광역시 동구 아양로 239 [35.89207, 128.64355]
경기 남양주시 화도읍 마석우리 [0, 0]
경기도 화성시 병점중앙로 10 10 [37.2088829, 127.0403629]
충북 단양군 대강면 장림리 [0, 0]
충청북도 청주시 청원구 율봉로 152 45-3 [0, 0]
서울 성북구 하월곡동 [37.60444, 127.03764]
충북 단양군 단양읍 도전리 632 [36.9855817, 128.3650067]
경기도 남양주시 경춘로 1291 (평내동) [37.6482974, 127.2360776]
광주광역시 서구 화정로 87-1 [35.1453053, 126.8742819]
충청남도 천안시 서북구 천안대로 1347 [36.9630845, 127.1228001]
세종특별자치시 조치원읍 세종로 2567  [36.607046, 127.2914653]
충청북도 청주시 상당구 상당로 13 15 [36.6344378, 127.4906003]
경기도 광명시 금하로 527 12 [0, 0]
전라북도 고창군 고창읍 전봉준로 88-15  [35.444742250000004, 126.68315418461583]
강원도 원주시 행구로 215 [37.34328, 127.980291]
광주광역시 북구 당뫼로 12 [35.167361, 126.9136047]
서울특별시 노원구 동일로 1229 [37.6175709, 127.075671]
부산광역시 수영구 연수로 249 12-18 [0, 0]
충청북도 청주시 청원구 오창읍 연구단지로 35  [36.7221336, 127.428769]
대구광역시 달서구 문화회관 3길 25 [0, 0]
충청북도 음성군 대소면 새미실길 74 [0, 0]
경기도 안산시 상록구 충장

In [84]:
out["lat_lng"] = la_ln
out["lat"] = [i[0] for i in out["lat_lng"]]
out["lng"] = [i[1] for i in out["lat_lng"]]

In [85]:
out[out["lat"]==0]

,기준년월,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터주소,지원센터상세주소,주소,지원센터주소2,지원센터주소3,lat_lng,lat,lng
37,201701,부산,기초,푸드뱅크,기장군푸드뱅크,부산 기장군 기장읍 대라리,175-7 우원빌딩 2층 202호,부산 기장군 기장읍 대라리 175-7 우원빌딩 2층 202호,부산 기장군 기장읍 대라리,부산 기장군 기장읍 대라리,"[0, 0]",0.0,0.0
74,201701,경기,광역,푸드뱅크,경기도광역푸드뱅크,경기도 수원시 장안구 서부로 2161,(89-5 6층),경기도 수원시 장안구 서부로 2161 (89-5 6층),경기도 수원시 장안구 서부로 2161,경기도 수원시 장안구 서부로 2161,"[0, 0]",0.0,0.0
76,201701,경기,기초,푸드뱅크,성남기초푸드뱅크,경기도 성남시 중원구 성남대로1151 20-7,(성남동),경기도 성남시 중원구 성남대로1151 20-7 (성남동),경기도 성남시 중원구 성남대로1151,경기도 성남시 중원구 성남대로 1151 20-7,"[0, 0]",0.0,0.0
77,201701,경기,기초,푸드뱅크,성남시청솔푸드뱅크,경기도 성남시 분당구 미금로 246,"(금곡동, 청솔마을주공6단지아파트)","경기도 성남시 분당구 미금로 246 (금곡동, 청솔마을주공6단지아파트)",경기도 성남시 분당구 미금로 246,경기도 성남시 분당구 미금로 246,"[0, 0]",0.0,0.0
80,201701,경기,기초,푸드뱅크,부천기초푸드뱅크,경기도 부천시 신흥로275 19,(중동),경기도 부천시 신흥로275 19 (중동),경기도 부천시 신흥로275,경기도 부천시 신흥로 275 19,"[0, 0]",0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,201701,전남,기초,푸드뱅크,해남푸드뱅크,전라남도 해남군 서림길 5,해남푸드뱅크,전라남도 해남군 서림길 5 해남푸드뱅크,전라남도 해남군 서림길 5,전라남도 해남군 서림길 5,"[0, 0]",0.0,0.0
427,201701,경기,기초,푸드뱅크,서평택푸드뱅크,경기도 평택시 안중읍 안중로8 23-11,1층,경기도 평택시 안중읍 안중로8 23-11 1층,경기도 평택시 안중읍 안중로8,경기도 평택시 안중읍 안중로 8 23-11,"[0, 0]",0.0,0.0
432,201701,경기,기초,푸드뱅크,희망등대푸드뱅크,경기도 성남시 수정구 희망로534 39,(산성동) 1층,경기도 성남시 수정구 희망로534 39 (산성동) 1층,경기도 성남시 수정구 희망로534,경기도 성남시 수정구 희망로 534 39,"[0, 0]",0.0,0.0
435,201701,경기,기초,푸드뱅크,화성은혜푸드뱅크,경기도 화성시 봉담읍 독정길 30,화성은혜푸드뱅크,경기도 화성시 봉담읍 독정길 30 화성은혜푸드뱅크,경기도 화성시 봉담읍 독정길 30,경기도 화성시 봉담읍 독정길 30,"[0, 0]",0.0,0.0


In [92]:
out.head()

,기준년월,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터주소,지원센터상세주소,주소,지원센터주소2,지원센터주소3,lat_lng,lat,lng
0,201701,중앙,전국,푸드뱅크,전국푸드뱅크,서울특별시 마포구 만리재로 14,한국사회복지회관 4층,서울특별시 마포구 만리재로 14 한국사회복지회관 4층,서울특별시 마포구 만리재로 14,서울특별시 마포구 만리재로 14,"[37.5440615, 126.951932]",37.544061,126.951932
1,201701,서울,광역,푸드뱅크,서울시광역푸드뱅크,서울 도봉구 창동,1-7,서울 도봉구 창동 1-7,서울 도봉구 창동,서울 도봉구 창동,"[37.64643, 127.04326]",37.646430,127.043260
2,201701,서울,기초,푸드뱅크,영등포사랑나눔푸드뱅크,서울특별시 영등포구 도림로61길 7,7,서울특별시 영등포구 도림로61길 7 7,서울특별시 영등포구 도림로61길 7,서울특별시 영등포구 도림로 61길 7,"[37.4983547, 126.9021709]",37.498355,126.902171
3,201701,서울,기초,푸드뱅크,성동희망푸드뱅크,서울특별시 성동구 왕십리로28길 10 준희빌딩,1층,서울특별시 성동구 왕십리로28길 10 준희빌딩 1층,서울특별시 성동구 왕십리로28길 10,서울특별시 성동구 왕십리로 28길 10 준희빌딩,"[37.5652258, 127.0318264]",37.565226,127.031826
4,201701,서울,기초,푸드뱅크,종로구푸드뱅크,서울 종로구 숭인1동,55-2 숭인1동주민센터 4층,서울 종로구 숭인1동 55-2 숭인1동주민센터 4층,서울 종로구 숭인1동,서울 종로구 숭인 1동,"[37.5776359, 127.01701239299508]",37.577636,127.017012


### 2. visualization

In [141]:
lat_,lng_ = out.loc[0]["lat"],out.loc[0]["lng"]
g_map = folium.Map(location=[lat_,lng_],zoom_start=18,tiles="Stamen Toner")
#icon = folium.features.CustomIcon('PASTE_URL', icon_size=(24, 24))

In [142]:
marker = folium.Marker(location=[lat_, lng_],popup="A",icon= folium.Icon(color="green",icon="star"))
marker.add_to(g_map)

In [143]:
col_list = ["red", "blue", "green", "purple", "orange", "darkred",
            "lightred", "beige", "darkblue", "darkgreen", "cadetblue",
            "darkpurple", "white", "pink", "lightblue", "lightgreen", "gray", "black", "lightgray"]

In [144]:
i = 0
for region in region_list:
    tar_ = region
    tar_df = out[out["지역코드"]==tar_]
    tar_df = tar_df[["지원센터단위코드","지원센터코드","lat","lng"]]
    tar_df["level_col"] = col_list[i]
    i += 1
    tar_df["shape"] = tar_df["지원센터단위코드"].map(lambda x: "home" if x=="기초" else ("flag"))
    for row,col in tar_df.iterrows():
        tar_col = col["level_col"]
        tar_pop = col["지원센터코드"]
        tar_shape = col["shape"]
        check_ = col["지원센터단위코드"]
        if check_ == "기초":
            marker = folium.Marker(location=[col["lat"], col["lng"]],popup=tar_pop,
                                   icon= folium.Icon(color=tar_col,icon=tar_shape)).add_to(g_map)
        else:
            marker = folium.Marker(location=[col["lat"], col["lng"]],popup=tar_pop,
                       icon= folium.Icon(color=tar_col,icon=tar_shape)).add_to(g_map)
            marker = folium.CircleMarker(location=[col["lat"], col["lng"]],popup=tar_pop
                                         ,radius=20
                                         ,icon= folium.Icon(color=tar_col,icon=tar_shape)).add_to(g_map)


In [145]:
g_map

In [ ]:
marker = folium.CircleMarker(location=[37.5440615, 126.951932],popup="A",radius=11,icon= folium.Icon(color="red",icon="flag"))
marker.add_to(g_map)
g_map

In [ ]:
outg_map = folium.Map(location=[37.5440615, 126.951932],zoom_start=18,tiles="Stamen Toner")
#icon = folium.features.CustomIcon('PASTE_URL', icon_size=(24, 24))

marker = folium.Marker(location=[37.5440615, 126.951932],popup="A",icon= folium.Icon(color="red",icon="flag"))
marker.add_to(g_map)
g_map

In [26]:
out["지원센터단위코드"].value_counts()

기초    424
광역     18
전국      1
Name: 지원센터단위코드, dtype: int64

In [30]:
out["통합시군구코드"].value_counts()# == 

경기      74
서울      62
충북      29
부산      29
전남      28
충남      27
인천      26
경남      24
경북      23
전북      22
강원      21
대구      21
광주      19
대전      17
울산      10
제주       6
세종       3
중앙       1
중앙물류     1
Name: 통합시군구코드, dtype: int64

## 기초센터-광역 연결 가정

- 광역센터의 같은 지역코드를 가진 기초센트는 연결되어 있다